In [35]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#database connection settings
import psycopg2

db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"

conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)

In [4]:
#############################  UTILITIES ###############################

import re
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_en = set(stopwords.words('english'))
stop_de = set(stopwords.words('german'))
stop_it = set(stopwords.words('italian'))

def pre_process(text):  
    try:
        text = text.lower()    
        tokens = word_tokenize(text)
        tokens = [t for t in tokens if t .isalpha()]    
        tokens = [t for t in tokens if t not in stop_en]
        tokens = [t for t in tokens if t not in stop_de]
        tokens = [t for t in tokens if t not in stop_it]
    except:
        tokens=[]
    return tokens

##############################################################################################

In [39]:
#############################  OPERA AMERICA CONTAINED IN NADAC ###############################
import pandas as pd

############################ database connection settings ####################################
import psycopg2
db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"
conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)
##############################################################################################


cursor = conn.cursor()  
query = "SELECT DISTINCT state, city, theatre from listings where source_id = 1"
cursor.execute(query)

tdf = pd.DataFrame(cursor.fetchall(), columns=['state', 'city', 'theatre'])

cursor = conn.cursor()  
query = "SELECT state, city, theatre, sum(performance_count) as listings from listings where source_id = 6 " + \
        "AND theatre not in (select name from theatre_mapping) " + \
        "GROUP BY state, city, theatre " + \
        "ORDER BY state, city, theatre"
cursor.execute(query)
l_df = pd.DataFrame(cursor.fetchall(), columns=['state', 'city', 'theatre', 'listings'])

################################ MAP TOKENS CONTAINED IN LISTINGS ##############################

import Levenshtein

map_dict = {}
oa_dict = {}
nadac_dict = {}

for idx2,t in tdf.iterrows():        
    oa_tokens = pre_process(t['theatre'])
    oa_dict[t['theatre']] = oa_tokens

for idx2,l in l_df.iterrows(): 
    comp_key = l['theatre']
    nadac_tokens = pre_process(l['theatre'])
    nadac_dict[comp_key] = [nadac_tokens, l['listings']]
    
print('Total records: ' + str(len(nadac_dict)))

for idx,key in enumerate(oa_dict):
#     if idx%100==0:
#         print(idx)

    map_dict[key] = {}        
    oa_tokens = oa_dict.get(key)
    
    for l_key in nadac_dict.keys():
        num_listings = nadac_dict.get(l_key)[1]
        if nadac_tokens:
            intersection = [value for value in oa_tokens if value in nadac_tokens]
            counter = len(intersection)
            map_dict[key][l_key] = [counter, counter/max(1, len(oa_tokens)), num_listings]                
            
# ################################################################################################  

# ########################### FIND MATCHES BASED ON HIGHEST % CONTAINED ##########################

closest_works={}

for idx,key in enumerate(list(map_dict.keys())):
            
    oa_dict = map_dict.get(key)
    if oa_dict.keys():
        key_min = max(oa_dict.keys(), key=(lambda k: oa_dict[k]))
        dist = oa_dict.get(key_min)[0]
        norm_dist = oa_dict.get(key_min)[1]
        num_listings = oa_dict.get(key_min)[2]
        closest_works[key] = [key_min, dist, norm_dist, num_listings]
    
cursor = conn.cursor()  
query = "SELECT DISTINCT state, city, theatre from listings where source_id = 1"
cursor.execute(query)

works_df = pd.DataFrame(cursor.fetchall(), columns=['state', 'city', 'theatre'])

closest_works

df = pd.DataFrame.from_dict(closest_works, orient='index', columns=['pm_theatre_nadac', 'pm_contained', 'pm_contained_perc', 'pm_num_listings'])
df['pm_theatre'] = df.index

df = df.merge(works_df, left_on=['pm_theatre'], right_on=['theatre'], how='inner')
df = df[['pm_theatre', 'pm_theatre_nadac', 'pm_contained', 'pm_contained_perc', 'pm_num_listings', 'theatre', 'city', 'state']]

df = df[df['pm_contained']!=0].sort_values(by=['pm_num_listings', 'pm_contained_perc', 'pm_theatre_nadac'], ascending=False)

# # df = df[(df['pm_num_listings'] < 2) & (df['pm_num_listings'] >= 1)]
# df = df[df['pm_contained_perc']==1]
# # df = df[df['pm_contained_perc']>=0.7]

# df = df[(df['pm_contained_perc']!=1) & (df['pm_contained_perc']>=0.7)]
# df = df[(df['pm_contained_perc']!=1) & (df['pm_contained_perc']>=0.7) & (df['pm_contained_perc']<0.8)]
# df = df[(df['pm_contained_perc']!=1) & (df['pm_contained_perc']>=0.6) & (df['pm_contained_perc']<0.7)]
# df = df[(df['pm_contained_perc']!=1) & (df['pm_contained_perc']>=0.5) & (df['pm_contained_perc']<0.6)]


# ###################################################################################################### 

# ###################################### DEALING WITH MAPPINGS #########################################

# df.head(100)

# ###################################################################################################### 

Total records: 4236


,pm_theatre,pm_theatre_nadac,pm_contained,pm_contained_perc,pm_num_listings,theatre,city,state
77,Miami Music Festival,Ohio Light Opera,2,0.666667,932,Miami Music Festival,Miami,Florida
21,Boston Early Music Festival,Ohio Light Opera,2,0.500000,932,Boston Early Music Festival,Cambridge,Massachusetts
44,Pine Mountain Music Festival,Ohio Light Opera,2,0.500000,932,Pine Mountain Music Festival,Hancock,Michigan
60,Festival Opera,Ohio Light Opera,1,0.500000,932,Festival Opera,Walnut Creek,California
115,Glimmerglass Festival,Ohio Light Opera,1,0.500000,932,Glimmerglass Festival,Cooperstown,New York
7,Florida Grand Opera,Ohio Light Opera,1,0.333333,932,Florida Grand Opera,Miami,Florida
24,Seagle Music Colony,Ohio Light Opera,1,0.333333,932,Seagle Music Colony,Schroon Lake,New York
29,Skylight Music Theatre,Ohio Light Opera,1,0.333333,932,Skylight Music Theatre,Milwaukee,Wisconsin
33,"Juventas Music, Inc.",Ohio Light Opera,1,0.333333,932,"Juventas Music, Inc.",Boston,Massachusetts
99,Pittsburgh Festival Opera,Ohio Light Opera,1,0.333333,932,Pittsburgh Festival Opera,Pittsburgh,Pennsylvania


In [14]:
# oa_dict
# nadac_dict
# map_dict